In [13]:
import pandas as pd
import json
import string

In [31]:
products = pd.read_csv('amazon_baby.csv')
products['review'].fillna('', inplace=True)

In [32]:
def remove_punctuation(text):
    table = str.maketrans({key: None for key in string.punctuation})
    return str(text).translate(table)

products['review_clean'] = products['review'].apply(remove_punctuation)
# remove the rating = 3 reviews
products = products[products['rating'] != 3].reset_index(drop=True)
# add labels
products['sentiment'] = products['rating'].apply(lambda rating : 1 if rating > 3 else -1)

In [33]:
products.head()

,name,review,rating,review_clean,sentiment
0,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
1,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
2,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
3,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
4,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [34]:
with open('week1_train_idx.json', 'r') as f:
    train_idx = json.load(f)
with open('week1_test_idx.json', 'r') as f:
    test_idx = json.load(f)

In [42]:
train_data, test_data = products.iloc[train_idx, :], products.iloc[test_idx, :]

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [48]:
# train a logistic regression model
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Quiz 1

In [59]:
sum(sentiment_model.coef_[0] >= 0)

87194

In [65]:
test_data.reset_index(inplace=True, drop=True)
sample_test_data = test_data[10:13]

In [77]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)

### Quiz 2

In [78]:
print(scores)

[  5.60129879  -3.17005306 -10.42408041]
